In [1]:
import re 
import pandas as pd
import nltk 

# Loading the training dataset

In [ ]:
comments = pb_read_csv('comments.csv')

In [ ]:
comments.head()

In [ ]:
comments.shape

In [ ]:
comments.dtypes

# Data Cleaning 

### 1. Droping null texts 


In [ ]:
comments.dropna(subset=['text'], inplace=True)

### 2. Removing punctuations and symbols

In [ ]:
training_set.text = training_set.text.apply(lambda x: re.sub(r'[^\w\s]', '', x) )

### 3. Removing stopwords

In [ ]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
training_set.text = training_set.text.apply(lambda x: ' '.join([word for word in nltk.word_tokenize(x) if word.lower() not in stopwords]))

### 4. Removing links 

In [ ]:
training_set.text = training_set.text.apply(lambda x: re.sub(r'\(?http\S+', '', x))

### 5. Lemmatization and Tokenizing

In [ ]:
from nltk.stem import PorterStemmer
porter_stemmer = PorterStemmer

lemmatizer = nltk.WordNetLemmatizer()
training_set.text = training_set.text.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(x)]))

In [ ]:
training_set.dropna(subset=['text'], inplace=True)
trainint_set = training_set[training_set.text != '']

## Data Exploration